In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#for building and training the U-Net model
from tensorflow.keras.layers import Conv2D, Input, Concatenate, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model

print("Loaded all libraries")

In [ ]:
train_fpath='../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/'
train_masks_fpath='../input/semantic-drone-dataset/RGB_color_image_masks/RGB_color_image_masks/'

#test_fpath='../input/3d_images/'
#test_masks_fpath='../input/3d_masks/'
print(os.listdir('../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/'))

In [ ]:
print("No. of images =",len(os.listdir(train_fpath)))
print("No. of image masks =",len(os.listdir(train_masks_fpath)))

In [ ]:
#1 example let's pick image id 'ID_0066_Z_0141'
def plot_img_and_mask(id):
    img = cv2.imread(train_fpath + id + '.png')
    img_mask = cv2.imread(train_masks_fpath + id + '.png')
    plt.figure(figsize=(10,10))
    plt.subplot(1,3,1,title='Actual image')
    plt.imshow(img)
    plt.subplot(1,3,3,title='Image mask')
    plt.imshow(img_mask)
    
plot_img_and_mask('173')

In [ ]:
plot_img_and_mask('173')

In [ ]:
def initialize_img_data(folder):
    lst=[]
    for image in os.listdir(folder):
    #load image in grayscale
        img=cv2.imread(folder+"/"+image,cv2.IMREAD_GRAYSCALE) 
        #convert to array
        img_array=Image.fromarray(img)
        # divide by 255
        resize_img=img_array.resize((128,128))
        norm_img=np.array(resize_img)/255
        # expand dimensions
        img_array=norm_img.reshape((128,128,1))
    
        lst.append(img_array)
    return lst

X=initialize_img_data(train_fpath)
y=initialize_img_data(train_masks_fpath)

In [ ]:
print(len(os.listdir(train_fpath)),len(X),len(y))
print(X[0].shape,y[0].shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,3,1,title='Actual image')
plt.imshow(X[10],cmap='gray')

plt.subplot(1,3,3,title='Image mask')
plt.imshow(y[10],cmap='gray')

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
x_train=X[:200]
y_train=y[:200]

x_test=X[200:]
y_test=y[200:]

In [ ]:
inputs=Input(shape=(128,128,1))

c1=Conv2D(8,(3,3), activation='relu', padding='same')(inputs)
c1=Conv2D(8,(3,3), activation='relu', padding='same')(c1)
p1=MaxPooling2D((2,2))(c1)

c2=Conv2D(16,(3,3), activation='relu', padding='same')(p1)
c2=Conv2D(16,(3,3), activation='relu', padding='same')(c2)
p2=MaxPooling2D((2,2))(c2)

c3=Conv2D(32,(3,3), activation='relu', padding='same')(p2)
c3=Conv2D(32,(3,3), activation='relu', padding='same')(c3)
p3=MaxPooling2D((2,2))(c3)

c4=Conv2D(64,(3,3), activation='relu', padding='same')(p3)
c4=Conv2D(64,(3,3), activation='relu', padding='same')(c4)
p4=MaxPooling2D((2,2))(c4)

c5=Conv2D(128,(3,3), activation='relu', padding='same')(p4)
c5=Conv2D(128,(3,3), activation='relu', padding='same')(c5)

u6=Conv2DTranspose(64,(2,2), strides=(2,2), padding='same')(c5)
u6=Concatenate()([u6,c4])  #optional
c6=Conv2D(64,(3,3), activation='relu', padding='same')(u6)
c6=Conv2D(64,(3,3), activation='relu', padding='same')(c6)

u7=Conv2DTranspose(32,(2,2), strides=(2,2), padding='same')(c6)
u7=Concatenate()([u7,c3])  #optional
c7=Conv2D(32,(3,3), activation='relu', padding='same')(u7)
c7=Conv2D(32,(3,3), activation='relu', padding='same')(c7)

u8=Conv2DTranspose(16,(2,2), strides=(2,2), padding='same')(c7)
u8=Concatenate()([u8,c2])  #optional
c8=Conv2D(16,(3,3), activation='relu', padding='same')(u8)
c8=Conv2D(16,(3,3), activation='relu', padding='same')(c8)

u9=Conv2DTranspose(8,(2,2), strides=(2,2), padding='same')(c8)
u9=Concatenate()([u9,c1])  #removed,axis=3
c9=Conv2D(8,(3,3), activation='relu', padding='same')(u9)
c9=Conv2D(8,(3,3), activation='relu', padding='same')(c9)

outputs=Conv2D(1,(1,1), activation='sigmoid')(c9)

In [ ]:
model=Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')

In [ ]:
x_train.shape

In [ ]:
early_stop=EarlyStopping(patience=5)
check_point=ModelCheckpoint('model.hdf5',save_best_only=True)
model.fit(x_train, y_train, epochs=200, callbacks=[early_stop, check_point])

In [ ]:
plt.figure(1, figsize=(15, 9))
n=0
for i in range(4):
    n+=1
    r=[i]
    plt.subplot(2,2,n)
    plt.subplots_adjust(hspace=0.5,wspace=0.5)
    #plt.imshow(x_test[r[0]],  cmap="gray") 
    plt.imshow(x_test[r[0]], alpha=0.5, cmap="Oranges") 
    
plt.suptitle(" images and the masks predictedby U-Net model highlighted in RED") 
plt.show()

In [ ]:
plt.figure(1, figsize=(15, 9))
n=0
for i in range(4):
    n+=1
    r=[i]
    plt.subplot(2,2,n)
    plt.subplots_adjust(hspace=0.5,wspace=0.5)
    #plt.imshow(x_test[r[0]],  cmap="gray") 
    plt.imshow(y_test[r[0]], alpha=0.5, cmap="Oranges") 
    
plt.suptitle("images and the masks predictedby U-Net model highlighted in RED") 
plt.show()